In [ ]:
from typing import Optional
from transformers import pipeline
from fastapi import FastAPI
from fastapi.responses import RedirectResponse
from langserve import add_routes
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.runnables import Runnable
from fastapi import Request

load_dotenv()
app = FastAPI()
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification",
                model="sinensia/distilbert-finetuned-imdb")


class TextRequest(BaseModel):
    text: str


label_map = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "POSITIVE"
}


@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")


class SentimentRunnable(Runnable):
    def __init__(self, model):
        self.model = model

    def invoke(self, input_text, config=None) -> dict:
        result = self.model(input_text)
        print(config)
        label = label_map.get(result[0]['label'], result[0]['label'])
        return {"label": label, "score": result[0]['score']}


class TestRunnable(Runnable):
    def __init__(self):
        pass

    def invoke(self, input: TextRequest, config=None) -> dict:

        return {"message": "Received", "input": input}


@app.post("/predict")
def predict_sentiment(request: TextRequest):
    return sentiment_runnable.invoke(request.text)


sentiment_runnable = SentimentRunnable(pipe)
test_runnable = TestRunnable()


add_routes(app, sentiment_runnable, path="/sentimiento")

add_routes(app, test_runnable, path="/test")

for route in app.router.routes:
    print(route.path)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)
